## Steps

### Import libraries

The first step is to import the libraries needed for this notebook.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
import os
import json

from IPython.display import display

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Connect to Wallaroo Instance

The following command will create a connection to the Wallaroo instance and store it in the variable `wl`.

In [2]:
# Login through local Wallaroo instance

# wl = wallaroo.Client()

# SSO login through keycloak

# wallarooPrefix = "YOUR PREFIX"
# wallarooSuffix = "YOUR PREFIX"

# wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
#                     auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
#                     auth_type="sso")

# os.environ["WALLAROO_SDK_CREDENTIALS"] = './creds.json.example'

# # wallarooPrefix = "docreleasetest"
# # wallarooSuffix = "wallaroocommunity.ninja"

wallarooPrefix = "docreleasetest"
wallarooSuffix = "wallaroocommunity.ninja"

# # wallarooPrefix = "sparkly-apple-3026"
# # wallarooSuffix = "wallaroo.community"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                     auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                     auth_type="sso")

# wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
#                     auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
#                     auth_type="sso")

Please log into the following URL in a web browser:

	https://docreleasetest.keycloak.wallaroocommunity.ninja/auth/realms/master/device?user_code=XNUJ-HLUU

Login successful!


In [4]:
import os
os.environ["ARROW_ENABLED"]="True"

### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace.

In [5]:
workspace_name = 'anomalytesting'

def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

{'name': 'anomalytesting', 'id': 4, 'archived': False, 'created_by': 'e8e34a3e-db75-49e3-a294-ac69f17bc177', 'created_at': '2023-05-03T21:10:11.609797+00:00', 'models': [], 'pipelines': []}

## Upload The Model

The housing model will be uploaded for use in our pipeline.

In [6]:
housing_model = wl.upload_model("anomaly-housing-model", "./models/housing.zip").configure("tensorflow")

### Build the Pipeline and Validation

The pipeline `anomaly-housing-pipeline` will be created and the `anomaly-housing-model` added as a step.  A validation will be created for outputs greater 100.0.  This is interpreted as houses with a value greater than $100 million with the `add_validation` method.  When houses greater than this value are detected, the `InferenceObject` will add it in the `check_failures` array with the message "price too high".

Once complete, the pipeline will be deployed and ready for inferences.


Two pipelines:

* Validation Integer:  A pipeline where the validation is set as an integer values.
* Validation float:  A pipeline where the validation is set as a float.

In [7]:
intpipeline = get_pipeline("intpipeline")
floatpipeline = get_pipeline("floatpipeline")

intpipeline.add_model_step(housing_model)
floatpipeline.add_model_step(housing_model)

intpipeline.add_validation('price too high', housing_model.outputs[0][0] < 100)
floatpipeline.add_validation('price too high', housing_model.outputs[0][0] < 100.0)

name,floatpipeline
created,2023-05-03 21:10:15.346191+00:00
last_updated,2023-05-03 21:10:15.346191+00:00
deployed,(none)
tags,
versions,b4fbce01-703d-4b9b-acda-4b19e0ab5da8
steps,


In [8]:
intpipeline.deploy()
floatpipeline.deploy()

name,floatpipeline
created,2023-05-03 21:10:15.346191+00:00
last_updated,2023-05-03 21:10:27.456869+00:00
deployed,True
tags,
versions,"9cf8bddb-0eb5-43fc-81ee-7a1ef5a63bf9, b4fbce01-703d-4b9b-acda-4b19e0ab5da8"
steps,anomaly-housing-model


In [9]:
test_input = pd.DataFrame.from_records({"dense_16_input":{"0":[0.02675675,0.0,0.02677953,0.0,0.0010046,0.00951931,0.14795322,0.0027145,0.03550877,0.98536841,0.02988655,0.04031725,0.04298041]}})

response_int = intpipeline.infer(test_input)
response_float = floatpipeline.infer(test_input)
display(response_int)
display(response_float)

,time,in.dense_16_input,out.dense_19,check_failures
0,2023-05-03 21:10:38.059,"[0.02675675, 0.0, 0.02677953, 0.0, 0.0010046, 0.00951931, 0.14795322, 0.0027145, 0.03550877, 0.98536841, 0.02988655, 0.04031725, 0.04298041]",[10.349834],1


,time,in.dense_16_input,out.dense_19,check_failures
0,2023-05-03 21:10:38.630,"[0.02675675, 0.0, 0.02677953, 0.0, 0.0010046, 0.00951931, 0.14795322, 0.0027145, 0.03550877, 0.98536841, 0.02988655, 0.04031725, 0.04298041]",[10.349834],0


In [10]:
test_input = pd.DataFrame.from_records({"dense_16_input":{"0":[0.02675675,0.0,0.02677953,0.0,0.0010046,0.00951931,0.14795322,0.0027145,2,0.98536841,0.02988655,0.04031725,0.04298041]}})

response_int = intpipeline.infer(test_input)
response_float = floatpipeline.infer(test_input)
display(response_int)
display(response_float)

,time,in.dense_16_input,out.dense_19,check_failures
0,2023-05-03 21:10:39.044,"[0.02675675, 0.0, 0.02677953, 0.0, 0.0010046, 0.00951931, 0.14795322, 0.0027145, 2.0, 0.98536841, 0.02988655, 0.04031725, 0.04298041]",[350.46994],1


,time,in.dense_16_input,out.dense_19,check_failures
0,2023-05-03 21:10:39.425,"[0.02675675, 0.0, 0.02677953, 0.0, 0.0010046, 0.00951931, 0.14795322, 0.0027145, 2.0, 0.98536841, 0.02988655, 0.04031725, 0.04298041]",[350.46994],1


### Undeploy The Pipeline

With the example complete, we undeploy the pipeline to return the resources back to the Wallaroo instance.

In [12]:
intpipeline.undeploy()
floatpipeline.undeploy()

name,floatpipeline
created,2023-05-03 21:10:15.346191+00:00
last_updated,2023-05-03 21:10:27.456869+00:00
deployed,False
tags,
versions,"9cf8bddb-0eb5-43fc-81ee-7a1ef5a63bf9, b4fbce01-703d-4b9b-acda-4b19e0ab5da8"
steps,anomaly-housing-model
